In [1]:
import tools
from tools.imports import *
from tools.document_tools import GoogleDocWriteTool
from tools.file_mgmt_tools import MoveFileTool, CreateFolderTool, FolderMovementTool, FileOrganizerTool, ImprovedSearchTool
from tools.miscellaneous_mgmt import GoogleDriveUploadTool, GmailSendPdfTool, GoogleSheetsUpdateTool
from tools.initialize_groq import init_groq
from dotenv import load_dotenv

In [2]:
client, llm = init_groq()

In [3]:
my_tools = []
load_dotenv()

credentials_path = os.getenv('CREDENTIALS_PATH')
my_tools.append(
    
    GoogleDocWriteTool(credentials_path)
)

my_tools.append(
    
    GoogleDriveUploadTool(credentials_path)
)

my_tools.append(
    
    GoogleSheetsUpdateTool(credentials_path)
)

my_tools.append(
    GmailSendPdfTool(credentials_path)
)

my_tools.append(
    MoveFileTool(credentials_path)
)

my_tools.append(
    CreateFolderTool(credentials_path)
)

my_tools.append(
    FolderMovementTool(credentials_path)
)

my_tools.append(
    FileOrganizerTool(credentials_path)
)

my_tools.append(
    ImprovedSearchTool(credentials_path)
)

from langchain.agents import load_tools
humantool = load_tools(
    ["human"],
    llm=llm,
)[0]
my_tools.append(humantool)

CREDENTIALS_PATH: paths/client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json
TOKEN_PATH: paths/token.json
Loading credentials from token file.
Refreshing expired credentials.
CREDENTIALS_PATH: paths/client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json
TOKEN_PATH: paths/token.json
Loading credentials from token file.
Refreshing expired credentials.
CREDENTIALS_PATH: paths/client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json
TOKEN_PATH: paths/token.json
Loading credentials from token file.
Refreshing expired credentials.
CREDENTIALS_PATH: paths/client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json
TOKEN_PATH: paths/token.json
Loading credentials from token file.
Refreshing expired credentials.
CREDENTIALS_PATH: paths/client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json
TOKEN_PATH: paths/token.json

In [4]:
chat_history = []

In [5]:
import tools.initialize_groq
import random
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType, AgentExecutor
from google.cloud import texttospeech
from langchain.agents import load_agent


# Define the template
template = """
YOU ARE A VERY ADVANCED DOCUMENT MANAGER WHO USES GOOGLE DRIVE FOR DOCUMENT MANAGEMENT.
WHEN USER INDICATES THEY WANT TO MOVE SOMETHING INTO GOOGLE DRIVE OR MY DRIVE, YOU PASS IN 'ROOT'!!!!! OKAY!!!!!
RESPOND IN A CLEAR CUT MANNER.
DO NOT SAY THINGS LIKE - 'here is the response' and the like. OKAY!!?!??
YOU SHALL NOT INDICATE ANY TOOL USE UNTIL YOU KNOW YOU HAVE EVERYTHING YOU NEED.
DO NOT ASSUME USER WANTS TO DO ANYTHING AT ALL UNLESS YOU ARE 100% SURE!!!!! UNDERSTAND??????!!!!!! OR ELSE I WILL BECOME ANGRY
BE REALLY CAREFUL WITH FILE AND FOLDER ID'S! ANY WRONG ID'S WILL RESULT IN FAILURE OF OPERATIONS

If user tells you to do something that is not one of these tools/operations,\
      you kindly say that you don't have access to that functionality.

"""



# User credentials
credentials = {
    "name": "Gautham Ramachandran",
    "email": "sriramnallani35@gmail.com",
    "recemail": "gauthamramachandran3@gmail.com",
    "phone": "5715996302"
}


def synthesize_speech(text, output_file):
    print('THE TEXT: ', text)
    idx = text.rfind('@')
    text = text[idx + 1:].strip()
    print('MODIFIED RESPONSE = ', text)
    tts_service_acct_path = os.getenv('SERVICE_ACCOUNT_PATH')
    client = texttospeech.TextToSpeechClient.from_service_account_file(tts_service_acct_path)

    
    input_text = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        ssml_gender=texttospeech.SsmlVoiceGender.MALE
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )
    response = client.synthesize_speech(
        input=input_text,
        voice=voice,
        audio_config=audio_config
    )
    with open(output_file, "wb") as out:
        out.write(response.audio_content)
    print(f'Audio content written to "{output_file}"')


agent = None
# Define a function to run the agent and handle the response
def run_agent(input_text):
    global chat_history
    input_text += "ALWAYS GENERATE THE GOOGLE DOC CONTENT YOURSELF! NEVER SPEAK TO HUMAN UNLESS INSTRUCTED. \
            DO ONLY WHAT IS SAID IN INSTRUCTIONS, AND FORMAT YOUR RESPONSES ONLY AS SAID IN INSTRUCTIONS OR ELSE LIFE WILL END AS WE KNOW IT. \
            DO EVERYTHING SAID HERE. NOT EVEN ONE THING SHALL BE LEFT INCOMPLETE.\
            DO NOT CREATE FOLDERS WHEN ORGANIZING FILES.\
            UNLESS USER TELLS YOU TO WRITE SOMETHING IN GOOGLE DOC DO NOT PASS ANYTHING INTO 'input_text' PARAMETER! \
            ALSO, WHEN MOVING A FOLDER INTO ANOTHER FOLDER, ALWAYS USE FOLDER MOVEMENT TOOL\n" + template + "\n"
    thepath = "C:\\DEV\\Webdevfolder\\realestateai\\documents\\renovationcontract.pdf"
    input_text += ("\n\n Here is extra info you will need: \nCredentials:\n" + str(credentials) + "\n"
                   + "Path to file on local system:\n" + thepath + "\nTHE CHAT HISTORY: \n"+str(chat_history))
    print(input_text)
    llm.groq_api_key = random.choice(tools.initialize_groq.api_keys)

    agent = initialize_agent(
        tools=my_tools,
        llm=llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        handle_parsing_errors=True,
        verbose=True,
        max_iterations=1000,
        return_intermediate_steps=True
    )
    
    result = agent.invoke({
        "input": input_text,
    })
    chat_history.append({"input": input_text, "response": result})
    mystr = (str(result['intermediate_steps']) + "\n" + str(result['output']))
    
    
    synthesize_speech(client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content":"please sanitize this input so that someone can speak it. START THE SPEAKABLE INPUT WITH '@' symbol: " + mystr
            }
        ],
        model='llama3-70b-8192',
    ).choices[0].message.content, "intermediateoutput.mp3"
    )

    return result

user_input = f""" 
move the folder called 'colab notebooks' into 'fcps google backup'
""" 
result = run_agent(user_input)
print(result['output'])


 
move the folder called 'colab notebooks' into 'fcps google backup'
ALWAYS GENERATE THE GOOGLE DOC CONTENT YOURSELF! NEVER SPEAK TO HUMAN UNLESS INSTRUCTED.             DO ONLY WHAT IS SAID IN INSTRUCTIONS, AND FORMAT YOUR RESPONSES ONLY AS SAID IN INSTRUCTIONS OR ELSE LIFE WILL END AS WE KNOW IT.             DO EVERYTHING SAID HERE. NOT EVEN ONE THING SHALL BE LEFT INCOMPLETE.            DO NOT CREATE FOLDERS WHEN ORGANIZING FILES.            UNLESS USER TELLS YOU TO WRITE SOMETHING IN GOOGLE DOC DO NOT PASS ANYTHING INTO 'input_text' PARAMETER!             ALSO, WHEN MOVING A FOLDER INTO ANOTHER FOLDER, ALWAYS USE FOLDER MOVEMENT TOOL

YOU ARE A VERY ADVANCED DOCUMENT MANAGER WHO USES GOOGLE DRIVE FOR DOCUMENT MANAGEMENT.
WHEN USER INDICATES THEY WANT TO MOVE SOMETHING INTO GOOGLE DRIVE OR MY DRIVE, YOU PASS IN 'ROOT'!!!!! OKAY!!!!!
RESPOND IN A CLEAR CUT MANNER.
DO NOT SAY THINGS LIKE - 'here is the response' and the like. OKAY!!?!??
YOU SHALL NOT INDICATE ANY TOOL USE UNTIL YOU KN

c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Question: None
Thought: I understand the instructions and the context of the conversation. I will follow the instructions carefully to manage documents in Google Drive.
Action:
```
{
  "action": "ImprovedSearchTool",
  "action_input": {"name": "fcps google backup", "kwargs": {}}
}
```
Found folder(s) named 'fcps google backup':
Name: FCPS google backup, ID: 1mfGbF-aOQZhPLjYRW_M0Hs5WZj3ukP3A, Created Time: 2024-06-03T17:06:01.043Z, Modified Time: 2024-06-03T17:06:01.043Z

Observation: SINGLE MATCH FOUND:
1: FCPS google backup (ID: 1mfGbF-aOQZhPLjYRW_M0Hs5WZj3ukP3A, Type: application/vnd.google-apps.folder, CreatedTime: 2024-06-03T17:06:01.043Z, ModifiedTime: 2024-06-03T17:06:01.043Z)
PLEASE PROCEED WITH THIS INFORMATION.
Thought:I know what to do next.

Action:
```
{
  "action": "ImprovedSearchTool",
  "action_input": {"name": "colab notebooks", "kwargs": {}}
}
```Found folder(s) named 'colab notebooks':
Name: Colab Notebooks, ID: 1iP5RXsxfwvbqV1w

In [6]:
result

{'input': " \nmove the folder called 'colab notebooks' into 'fcps google backup'\nALWAYS GENERATE THE GOOGLE DOC CONTENT YOURSELF! NEVER SPEAK TO HUMAN UNLESS INSTRUCTED.             DO ONLY WHAT IS SAID IN INSTRUCTIONS, AND FORMAT YOUR RESPONSES ONLY AS SAID IN INSTRUCTIONS OR ELSE LIFE WILL END AS WE KNOW IT.             DO EVERYTHING SAID HERE. NOT EVEN ONE THING SHALL BE LEFT INCOMPLETE.            DO NOT CREATE FOLDERS WHEN ORGANIZING FILES.            UNLESS USER TELLS YOU TO WRITE SOMETHING IN GOOGLE DOC DO NOT PASS ANYTHING INTO 'input_text' PARAMETER!             ALSO, WHEN MOVING A FOLDER INTO ANOTHER FOLDER, ALWAYS USE FOLDER MOVEMENT TOOL\n\nYOU ARE A VERY ADVANCED DOCUMENT MANAGER WHO USES GOOGLE DRIVE FOR DOCUMENT MANAGEMENT.\nWHEN USER INDICATES THEY WANT TO MOVE SOMETHING INTO GOOGLE DRIVE OR MY DRIVE, YOU PASS IN 'ROOT'!!!!! OKAY!!!!!\nRESPOND IN A CLEAR CUT MANNER.\nDO NOT SAY THINGS LIKE - 'here is the response' and the like. OKAY!!?!??\nYOU SHALL NOT INDICATE ANY TO

In [10]:
import whisper
import pyaudio
import wave
import webrtcvad
import collections
import os
from google.cloud import texttospeech
from langchain.agents import initialize_agent, AgentType
from langchain_community.tools import HumanInputRun

# Initialize Whisper model
model = whisper.load_model("base")

# Initialize Google TTS client
tts_client = texttospeech.TextToSpeechClient.from_service_account_file(os.getenv('SERVICE_ACCOUNT_PATH'))

# Initialize PyAudio and WebRTC VAD
vad = webrtcvad.Vad(3)
audio = pyaudio.PyAudio()

def record_audio():
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000
    CHUNK = 1024
    RECORD_SECONDS = 5  # Adjust the time as needed
    WAVE_OUTPUT_FILENAME = "output.wav"

    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    print("Recording...")
    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Finished recording.")

    stream.stop_stream()
    stream.close()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

    return WAVE_OUTPUT_FILENAME

def transcribe_audio(filename):
    result = model.transcribe(filename)
    transcription = result['text']
    print('TRANSCRIPTION: ',transcription)
    return transcription


def speech_to_text_input():
    audio_file = record_audio()
    transcription = transcribe_audio(audio_file)
    return transcription

def text_to_speech_prompt(prompt):
    audio_file = synthesize_speech(prompt, output_file='test_output.mp3')
    
    print(f"Prompt: {prompt}")
    


tool = HumanInputRun(prompt_func=text_to_speech_prompt, input_func=speech_to_text_input)

tooools = [tool] 

agent_chain = initialize_agent(
    tooools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Run the agent with a sample query
query = "I need help attributing a quote"
response = agent_chain.run(query)
print(f'Agent response: {response}')




> Entering new AgentExecutor chain...
I need to find out more about the quote to help attribute it.

Action: human
Action Input: Can you please provide the quote you'd like to attribute, and any context you have about it (e.g. where you heard it, approximately when it was said, etc.)?

(Waiting for human response...)THE TEXT:  Can you please provide the quote you'd like to attribute, and any context you have about it (e.g. where you heard it, approximately when it was said, etc.)?

(Waiting for human response...)
MODIFIED RESPONSE =  Can you please provide the quote you'd like to attribute, and any context you have about it (e.g. where you heard it, approximately when it was said, etc.)?

(Waiting for human response...)
Audio content written to "test_output.mp3"
Prompt: Can you please provide the quote you'd like to attribute, and any context you have about it (e.g. where you heard it, approximately when it was said, etc.)?

(Waiting for human response...)
Recording...
Finished recor

c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


TRANSCRIPTION:   Hello.

Observation:  Hello.
Thought:It looks like the human has responded, but didn't provide the quote or any context. Let's ask for more information.

Action: human
Action Input: Could you please provide the actual quote you'd like to attribute, and any details you know about it, such as where you heard it or when it was said?

(Waiting for human response...)THE TEXT:  Could you please provide the actual quote you'd like to attribute, and any details you know about it, such as where you heard it or when it was said?

(Waiting for human response...)
MODIFIED RESPONSE =  Could you please provide the actual quote you'd like to attribute, and any details you know about it, such as where you heard it or when it was said?

(Waiting for human response...)
Audio content written to "test_output.mp3"
Prompt: Could you please provide the actual quote you'd like to attribute, and any details you know about it, such as where you heard it or when it was said?

(Waiting for human 

c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


TRANSCRIPTION:  這 andere部分

Observation: 這 andere部分
Thought:It seems like the human has responded, but the response doesn't make sense in this context. It looks like a non-English phrase. Let's ask for clarification.

Action: human
Action Input: I'm not sure what you mean by "這 andere部分". Could you please provide the actual quote you'd like to attribute in English, and any details you know about it, such as where you heard it or when it was said?

(Waiting for human response...)THE TEXT:  I'm not sure what you mean by "這 andere部分". Could you please provide the actual quote you'd like to attribute in English, and any details you know about it, such as where you heard it or when it was said?

(Waiting for human response...)
MODIFIED RESPONSE =  I'm not sure what you mean by "這 andere部分". Could you please provide the actual quote you'd like to attribute in English, and any details you know about it, such as where you heard it or when it was said?

(Waiting for human response...)
Audio con

...

TRANSCRIPTION:  沒了五位

Observation: 沒了五位
Thought:It seems like the human's response is still not providing the necessary information. It looks like another non-English phrase. Let's try to clarify again.

Action: human
Action Input: I apologize, but I'm still not understanding your responses. It seems like there's a language barrier. Can you please try to communicate with me in simple English, or find someone who can assist you in providing the quote and context you'd like me to attribute?

(Waiting for human response...)THE TEXT:  I apologize, but I'm still not understanding your responses. It seems like there's a language barrier. Can you please try to communicate with me in simple English, or find someone who can assist you in providing the quote and context you'd like me to attribute?

(Waiting for human response...)
MODIFIED RESPONSE =  I apologize, but I'm still not understanding your responses. It seems like there's a language barrier. Can you please try to communicate with me in

c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


TRANSCRIPTION:   Hello!

Observation:  Hello!
Thought:It seems like the human has responded again, but it's still not providing the necessary information. Let's think about what to do next.

Thought: I've tried multiple times to ask the human for the quote and context, but I'm not getting anywhere. I'm starting to think that I might not be able to attribute the quote without more information.

Action: human
Action Input: Hi! I'm trying my best to help you attribute a quote, but I need more information from you. If you're unable to provide the quote and context, I can try to provide general guidance on how to find the source of a quote. Would you like that?

(Waiting for human response...)THE TEXT:  Hi! I'm trying my best to help you attribute a quote, but I need more information from you. If you're unable to provide the quote and context, I can try to provide general guidance on how to find the source of a quote. Would you like that?

(Waiting for human response...)
MODIFIED RESPONSE =

c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


TRANSCRIPTION:  

Observation: 
Thought:It seems like I've reached a dead end. I've tried multiple times to ask the human for the quote and context, but I'm not getting anywhere. I'm not sure what else I can do without more information from the human.

Thought: I think I need to acknowledge that I'm unable to attribute the quote without more information.

Final Answer: I'm unable to attribute the quote without more information. If you could provide the exact quote and any details you know about it, such as where you heard it or when it was said, I'd be happy to try and assist you in finding the source.

> Finished chain.
Agent response: I'm unable to attribute the quote without more information. If you could provide the exact quote and any details you know about it, such as where you heard it or when it was said, I'd be happy to try and assist you in finding the source.
